In [1]:
import dgl
import torch
import dgl.data
import networkx as nx
import metis
import matplotlib.pyplot as plt
import numpy as np
from models import *
from train import *

In [74]:
compression_rate = "big"
model = "GCN"
dataset = "arxiv"
k = 20

dataset_nx, dataset_dgl, data = load_dataset(dataset)
labels = dataset_dgl.ndata['label']
train_ids = (dataset_dgl.ndata['train_mask']==True).nonzero()
val_ids = (dataset_dgl.ndata['val_mask']==True).nonzero()
nx.set_node_attributes(dataset_nx,0,"class")
nx.set_node_attributes(dataset_nx,5,"set")
nx.set_node_attributes(dataset_nx,"None","pred")

for partition in range(k):
    if compression_rate == "teacher":
        checkpoint_path = "saved_models/best_"+str(model)+"_"+str(dataset)+"_"+"p"+str(partition+1)+"_k"+str(k)+".pth"
    checkpoint = torch.load(checkpoint_path)

    # get the partition node ids to create the subgraph
    sg_node_ids = checkpoint['node_ids'].cpu()
    sg_nx = dataset_nx.subgraph(sg_node_ids.tolist())

    args = checkpoint['args']
    # create a gnn for this partition using graph parameters
    sg = dgl.node_subgraph(dataset_dgl, sg_node_ids)
    features = sg.ndata['feat']
    # load the model
    m = load_student_model(args.gnn,features,data.num_classes,args.heads, args.dropout, args.compression_rate)
    if compression_rate == "teacher":
        m = load_model(args.gnn,features,data.num_classes,args.heads, args.dropout)
    m.load_state_dict(checkpoint['model_state_dict'])
    m.eval()

    # forward pass for evaluation
    with torch.no_grad():
        # Forward
        logits = m(sg, features)
        pred = logits.argmax(1)

        # store the predictions
        # preds[sg_node_ids] = pred
        # val = sg.ndata['val_mask']
        # print("partition",partition,"acc:",sum(pred[val]==sg.ndata['label'][val])/len((val==True).nonzero()))
    # print(len(sg_node_ids))
    pred_dict = { i.item() : str(pred[idx].item()) for idx,i in enumerate(sg_node_ids)}
    class_dict = { i.item() : labels[i].item() for i in sg_node_ids}
    set_dict = {}
    for i in sg_node_ids:
        if i in train_ids:
            set_dict[i.item()] = 50
        elif i in val_ids:
            set_dict[i.item()] = 20
        else:
            set_dict[i.item()] = 2
    # print(set_dict)
    nx.set_node_attributes(sg_nx,class_dict,"class")
    nx.set_node_attributes(sg_nx,set_dict,"set")
    nx.set_node_attributes(sg_nx,pred_dict,"pred")
    nx.write_gml(sg_nx,"arxiv"+str(partition)+".gml")
    # print(sg_nx.nodes[14]['Label'])
# train_dict = { i.item() : 50 for i in train_ids}
# val_dict = { i.item() : 20 for i in val_ids}
# nx.set_node_attributes(dataset_nx,train_dict,"set")
# nx.set_node_attributes(dataset_nx,val_dict,"set")
# nx.write_gml(dataset_nx,"cora.gml")


Number of categories: 40


FileNotFoundError: [Errno 2] No such file or directory: 'saved_models/best_bigGCN_arxiv_p1_k20.pth'